Try an extremely simple model: two circles, sampled from two populations, and try to scope out the oddities in the mixture model.

In [ ]:
# needed on macs due to subtle multiprocessing differences used in dynesty
import sys
import multiprocessing
if sys.platform == 'darwin':
    multiprocessing.set_start_method('fork')

njobs = multiprocessing.cpu_count()  # might need to be modified for optimal behavior on a specific machine - e.g. computers with fast and slow cores
njobs

In [ ]:
import math

import numpy as np
np.seterr(invalid='ignore')

from scipy import stats

from tqdm.notebook import tqdm

In [ ]:
from matplotlib import pyplot as plt
from IPython.display import display
%matplotlib agg

In [ ]:
def produce_uniform_circle(outerr, density):
    x, y = np.random.rand(2, int(density*4*outerr**2)) * outerr*2 - outerr
    r = np.hypot(x,y)
    msk = r< outerr
    return np.array((x[msk], y[msk]))

In [ ]:
def loglikelihood(rinner, fouter, router, data):
    x, y = data
    r = np.hypot(x, y)
    
    innerarea = np.pi*rinner**2
    outerarea = np.pi*router**2
    lfg0 = (1-fouter)/innerarea
    lbkg0 = fouter/outerarea

    return np.sum(np.log(lfg0*(r<=rinner) + lbkg0*len(r)), axis=-1)

In [ ]:
rinnertrue = 1
router = 3

fg = produce_uniform_circle(rinnertrue, 25)
bg = produce_uniform_circle(router, 2)
both = np.concatenate((fg, bg), axis=1)

ftrue = bg.shape[1]/both.shape[1]

ris = np.linspace(0.1, 2, 100000)
fouter = np.linspace(0, 1, 11)[1:]
rg, fgg = np.meshgrid(ris, fouter)

lg = loglikelihood(rg[..., np.newaxis], fgg[..., np.newaxis], router, both)

fig = plt.figure(figsize=(8,8))
plt.pcolor(rg, fgg, lg)
plt.axvline(rinnertrue, c='r', ls=':')
plt.axhline(ftrue, c='r', ls=':')
plt.xlabel('rinner')
plt.ylabel('fouter')
plt.colorbar().set_label('log like')
display(fig)
plt.close()


fig = plt.figure(figsize=(8,8))

for idx in [0, lg.shape[0]//2, -2, -1]:
    mx = np.max(lg[idx])
    mi = np.min(lg[idx])
    m = np.mean(lg[idx])
    lscaled = (lg[idx] - mi)/(mx-mi)
    p = plt.plot(rg[idx], lscaled, label=f'{idx}, mean={m}')
#plt.plot(rg[idx], np.pi/(np.pi*rg[idx]**2), ls=':', lw=1, c='k', label='$r^{-2}$')
plt.axvline(rinnertrue, c='r', ls=':')
plt.ylim(-0.01, 1.01)
plt.legend(loc=0)

display(fig)
plt.close()

In [ ]:
rinnertrue = 1
router = 3

fg = produce_uniform_circle(rinnertrue, 250)
bg = produce_uniform_circle(router, 20)
both = np.concatenate((fg, bg), axis=1)

ftrue = bg.shape[1]/both.shape[1]

ris = np.linspace(0.1, 2, 100000)
fouter = np.linspace(0, 1, 11)[1:]
rg, fgg = np.meshgrid(ris, fouter)

lg = loglikelihood(rg[..., np.newaxis], fgg[..., np.newaxis], router, both)

fig = plt.figure(figsize=(8,8))
plt.pcolor(rg, fgg, lg)
plt.axvline(rinnertrue, c='r', ls=':')
plt.axhline(ftrue, c='r', ls=':')
plt.xlabel('rinner')
plt.ylabel('fouter')
plt.colorbar().set_label('log like')
display(fig)
plt.close()


fig = plt.figure(figsize=(8,8))

for idx in [0, lg.shape[0]//2, -2, -1]:
    mx = np.max(lg[idx])
    mi = np.min(lg[idx])
    m = np.mean(lg[idx])
    lscaled = (lg[idx] - mi)/(mx-mi)
    p = plt.plot(rg[idx], lscaled, label=f'{idx}, mean={m}')
#plt.plot(rg[idx], np.pi/(np.pi*rg[idx]**2), ls=':', lw=1, c='k', label='$r^{-2}$')
plt.axvline(rinnertrue, c='r', ls=':')
plt.ylim(-0.01, 1.01)
plt.legend(loc=0)

display(fig)
plt.close()

Compare to a binned approach

In [ ]:
r = np.hypot(*both)
n, be = np.histogram(r, bins=100)
bm = (be[1:]+be[:-1])/2
darea = 2*np.pi*np.diff(be)*bm

fig = plt.figure()
plt.step(bm, n/darea)
plt.close();fig

In [ ]:
ris = np.linspace(0.1, 2, 10000)
fouter = np.linspace(0, 1, 21)[1:]

In [ ]:
rg, fgg = np.meshgrid(ris, fouter)
rg = rg[..., np.newaxis]
fgg = fgg[..., np.newaxis]

totaldensity = r.shape[0]/(np.pi*router**2)
outerdensity = totaldensity*fgg
innerdensity = totaldensity*(1-fgg)

npred = darea*innerdensity*(bm<=rg) + outerdensity*darea
probgrid = stats.poisson(mu=npred).logpmf(n)
probgrid.shape

In [ ]:
llike = np.sum(probgrid, axis=-1)

fig = plt.figure(figsize=(10, 10))
plt.pcolor(rg[..., 0], fgg[..., 0], np.exp(llike/probgrid.shape[-1]))
plt.colorbar().set_label('log like')
plt.axvline(rinnertrue, c='r', ls=':')
plt.axhline(ftrue, c='r', ls=':')

plt.close();fig